In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
from scipy.stats import chisquare
from PIL import Image
from __future__ import print_function
import torch
from torchvision import transforms as T
from torchvision import models
import csv

In [ ]:
path_jpg = 'C:/Users/bolub/OneDrive/Masaüstü/Ders/Year_4/FYP/pad-ufes-20/images/imgs_part_1/imgs_part_1_process'
img_files = [x for x in os.listdir(path_jpg)]
print(img_files)

In [ ]:
def preProcess(image):
    #Read the image and convert from BGR to RGB format
    img = cv.imread(image)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    #Resize the image
    img = cv.resize(img, (200, 200))
    
    #Apply the Gaussian Blur
    img_blur = cv.GaussianBlur(img, (9,9), 0)
    
    #Apply the Morphological Opening and Closing Operations
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(img_blur, cv.MORPH_OPEN, kernel)
    img_closed = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel)
    
    #Apply CLAHE
    gray = cv.cvtColor(img_closed, cv.COLOR_RGB2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    
    # Thresholding operation
    _, threshold = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    
    # Morphological Opening and Closing
    kernel_2 = np.ones((5,5),np.uint8)
    opening_thresh = cv.morphologyEx(threshold, cv.MORPH_OPEN, kernel_2)
    threshold_closed = cv.morphologyEx(opening_thresh, cv.MORPH_CLOSE, kernel_2)
    
    # Find Contours
    contours, _ = cv.findContours(threshold_closed, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #Sort the Contours and Define the largest one by areas        
    sorted_contours = sorted(contours, key=cv.contourArea, reverse=True)
    # Define the empty mask for contours
    mask = np.zeros(gray.shape, np.uint8)
    # Draw the contours on top of the mask, take the largest five
    cv.drawContours(mask, sorted_contours, 0, (255, 255, 255), -1)
    
    # Apply the mask to the original lesion image
    result = cv.bitwise_and(img, img, mask=mask)

    return result , sorted_contours

In [ ]:
def drawMinAreaRect(image, contours):
    
    #draw minimum area rectangle around the contour
    rect = cv.minAreaRect(contours[0])
    
    #Get the angle of the rectangle with respect to bottom right point
    angle_rect = rect[2]
    actual_angle = 0
    
    #Check if the rectangle angle is lower than -45
    if abs(angle_rect) <= 45:
        actual_angle = abs(angle_rect)
    else:
        actual_angle = 90 - abs(angle_rect)
    
    #Define the four corners of the rectangle
    box = cv.boxPoints(rect)
    box = np.intp(box)
    
    #Get the center of the bounding rectangle for rotation matrix
    center_y = round(int(rect[0][1]))
    center_x = round(int(rect[0][0]))
    
    x_img, y_img = image.shape[:2]
    
    # Define the translation offsets
    tx = 100 - center_x
    ty = 100 - center_y
    
    # Define the transformation matrix
    M_translate  = np.array([
    [1, 0, tx],
    [0, 1, ty]
    ], dtype=np.float32)
    
    #Define the rotation matrix and necessary offsets, both for larger than 45 and smaller than 45
    M = cv.getRotationMatrix2D((center_x, center_y), actual_angle, 1)
    M2 = cv.getRotationMatrix2D((center_x, center_y), -actual_angle, 1)
    
    #Apply the rotation and transformation matrices to the image and the rectangle
    if angle_rect < 45:
        img_rot = cv.warpAffine(image, M, (x_img, y_img))
        img_rot_trans = cv.warpAffine(img_rot, M_translate, (x_img, y_img))
        box_rot = cv.transform(np.array([box]), M)
        box_rot_trans = cv.transform(box_rot, M_translate)
        img_plot = img_rot_trans
    else:
        img_rot = cv.warpAffine(image, M2, (x_img, y_img))
        img_rot_trans = cv.warpAffine(img_rot, M_translate, (x_img, y_img))
        box_rot = cv.transform(np.array([box]), M2)
        box_rot_trans = cv.transform(box_rot, M_translate)
        img_plot = img_rot_trans
    
    return img_rot_trans

In [ ]:
#Find Asymmetry

def find_asymmetry(rotated_image):
    gray_rotated_image = cv.cvtColor(rotated_image, cv.COLOR_RGB2GRAY)
    _ , thresh = cv.threshold(gray_rotated_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    # Get the image dimensions
    height, width = gray_rotated_image.shape
    
    # Calculate the histograms for colour asymmetry
    r,g,b = cv.split(rotated_image)
    hist_r_left = cv.calcHist([r[:,:width//2]], [0], None, [256], [0, 256])
    hist_g_left = cv.calcHist([g[:,:width//2]], [0], None, [256], [0, 256])
    hist_b_left = cv.calcHist([b[:,:width//2]], [0], None, [256], [0, 256])
    hist_r_right = cv.calcHist([r[:width//2,:]], [0], None, [256], [0, 256])
    hist_g_right = cv.calcHist([g[:width//2,:]], [0], None, [256], [0, 256])
    hist_b_right = cv.calcHist([b[:width//2,:]], [0], None, [256], [0, 256])
    
    # Get the Correlation Metric value of Histograms, for each channel
    metric_val_r = cv.compareHist(hist_r_left, hist_r_right, cv.HISTCMP_CORREL)
    metric_val_g = cv.compareHist(hist_g_left, hist_g_right, cv.HISTCMP_CORREL)
    metric_val_b = cv.compareHist(hist_b_left, hist_b_right, cv.HISTCMP_CORREL)
    metric = (metric_val_r + metric_val_g + metric_val_b)/3
    
    # Decide on the colour asymmetry score
    asymmetry_score_color = 0
    
    if metric < 0.1:
        asymmetry_score_color = 0
    elif metric > 0.1 and metric < 0.2:
        asymmetry_score_color = 1
    else:
        asymmetry_score_color = 2
    
    # Fold the image along the vertical axis
    left_half = thresh[:, :width // 2]
    right_half = np.flip(thresh[:, width // 2:], axis=1)
    folded_image_vertical = np.abs(left_half - right_half)
    
    #Fold the image along the horizontal axis
    top_half = thresh[:height // 2, :]
    bottom_half = np.flip(thresh[:height // 2,:], axis=1)
    folded_image_horizontal = np.abs(top_half - bottom_half)

    # Find non-zero pixel values in the folded image
    nonzero_pixels = rotated_image[np.nonzero(rotated_image)]
    nonzero_pixels_vertical = folded_image_vertical[np.nonzero(folded_image_vertical)]
    nonzero_pixels_horizontal = folded_image_horizontal[np.nonzero(folded_image_horizontal)]

    # Count the number of non-zero pixel values
    num_nonzero_pixels = len(nonzero_pixels)
    num_nonzero_pixels_vertical = len(nonzero_pixels_vertical)
    num_nonzero_pixels_horizontal = len(nonzero_pixels_horizontal)
    
    #Decide on the score for asymmetry
    asymmetry_pixels_shape = (((num_nonzero_pixels_horizontal*0.5) + (num_nonzero_pixels_vertical*0.5)) / num_nonzero_pixels)*100
    
    if asymmetry_pixels_shape < 5:
        asymmetry_score_shape = 0
    elif asymmetry_pixels_shape > 5 and asymmetry_pixels_shape < 10:
        asymmetry_score_shape = 1
    else:
        asymmetry_score_shape = 2
    
    # Get the final asymmetry score
    asymmetry_score = (asymmetry_score_shape + asymmetry_score_color)/2
    
    # Return the final asymmetry score
    return asymmetry_score*1.3

In [ ]:
#Finding border irregularities

def find_border_irregularity(rotated_image):
    
    #Get dimensions of the image
    width = rotated_image.shape[1]
    height = rotated_image.shape[0]
    
    # Convert the image to grayscale
    gray_image = cv.cvtColor(rotated_image, cv.COLOR_BGR2GRAY)

    # Threshold the grayscale image to obtain a binary image
    _, binary_image = cv.threshold(gray_image, 0, 255, cv.THRESH_BINARY)
    
    # Find Contours
    contours, _ = cv.findContours(binary_image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    #Sort the Contours and Define the largest one by areas        
    sorted_contours = sorted(contours, key=cv.contourArea, reverse=True)

    # Define a minimum enclosing circle
    (x, y), radius = cv.minEnclosingCircle(sorted_contours[0])
    center_circ = (int(x), int(y))
    radius = int(radius)
    cv.circle(rotated_image, center_circ, radius, (255, 0, 0), 2)
    
    # Define the translation offsets
    tx = 100 - center_circ[0]
    ty = 100 - center_circ[1]
    
    # Define the transformation matrix
    M_translate  = np.array([
    [1, 0, tx],
    [0, 1, ty]
    ], dtype=np.float32)
    
    #Shift the lesion to the center so that circle is aligned with each sector
    img_shifted = cv.warpAffine(binary_image, M_translate, (width, height))
    
    # Find Contours
    contours_shifted, _ = cv.findContours(img_shifted, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    sorted_shifted_contours = sorted(contours_shifted, key=cv.contourArea, reverse=True)
    
    #Calculate distance of each contour against the center of the circle
    dist_arr = []
    
    for i in range(len(sorted_shifted_contours[0])):
        contour_x = int(sorted_shifted_contours[0][i][0][0])
        contour_y = int(sorted_shifted_contours[0][i][0][1])
        dist = np.linalg.norm(np.array([100, 100]) - np.array([contour_x, contour_y]))
        dist_arr.append(dist)
    
    # Find the difference and hence, the score for all sectors
    sum_dist = sum(dist_arr)
    sum_circle = radius * int(len(dist_arr))
    dissimilarity_index = 1 - (sum_dist / sum_circle)
    
    # Decide border irregularity score
    border_score = 0
    
    if dissimilarity_index < 0.05:
        border_score = 0
    elif dissimilarity_index > 0.05 and dissimilarity_index < 0.1:
        border_score = 1
    elif dissimilarity_index > 0.1 and dissimilarity_index < 0.15:
        border_score = 2
    elif dissimilarity_index > 0.15 and dissimilarity_index < 0.2:
        border_score = 3
    elif dissimilarity_index > 0.2 and dissimilarity_index < 0.25:
        border_score = 4
    elif dissimilarity_index > 0.25 and dissimilarity_index < 0.3:
        border_score = 5
    elif dissimilarity_index > 0.3 and dissimilarity_index < 0.35:
        border_score = 6
    elif dissimilarity_index > 0.35 and dissimilarity_index < 0.4:
        border_score = 7
    elif dissimilarity_index > 0.4:
        border_score = 8
        
    return border_score*0.1

In [ ]:
#Finding Color Number

def check_color(image_path):
    
    colour_codes = np.array([[1.0, 1.0, 1.0],[0.0, 0.0, 0.0],
                             [0.0, 0.0, 1.0],[0.25, 0.52, 0.80],
                             [0.13, 0.26, 0.40],[0.54, 0.52, 0.0]])
    count = np.array([0,0,0,0,0,0])
    
    img = cv.imread(image_path)
    img = cv.GaussianBlur(img, (5,5),0)
    img = cv.resize(img, (200, 200))
    
    #Set the thresholds for control, T for each pixel, L for the whole appearance of the colour
    L = 0.05
    T = 0.4
    score = 0
    
    #Loop through each pixel on the image
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            
            b = img[y,x,0] / 255
            g = img[y,x,1] / 255
            r = img[y,x,2] / 255
            
            for i in range(6):
                
                D = np.sqrt((b - colour_codes[i][0])**2 + (g - colour_codes[i][1])**2 + (r - colour_codes[i][2])**2)
                if D < T:
                    count[i] += 1
    
    percentages = count / (img.shape[0]*img.shape[1])
    
    for j in range(len(percentages)):
        if percentages[j] > L:
            score += 1
    
            
    return score*0.5, percentages

In [ ]:
file = open("Diameter_info.csv", "r")
data_diameter = list(csv.reader(file, delimiter=","))
file.close()

score_diameter = []

for i in range(len(data_diameter)):
    
    if data_diameter[i][0].isnumeric() == True:
        if int(data_diameter[i][0]) < 6:
            score_diameter.append(0.5* int(data_diameter[i][0]))
        else:
            score_diameter.append(2.5)
    else:
        score_diameter.append(1)

asym_arr = []
border_arr = []
color_arr = []


for i in range(len(img_files)):

    input_var = path_jpg + '/' + img_files[i]

    result, contours = preProcess(input_var)   
    rot_img = drawMinAreaRect(result, contours)
    score_asym = find_asymmetry(rot_img)
    score_border = find_border_irregularity(rot_img)
    score_color, _ = check_color(input_var)
    
    asym_arr.append(score_asym)
    border_arr.append(score_border)
    color_arr.append(score_color)

# Retrieve final score

score_final = []

for i in range(len(img_files)):
    s = asym_arr[i] + border_arr[i] + color_arr[i] + score_diameter[i]
    score_final.append(s)


output_rows = zip(asym_arr, border_arr, color_arr, score_diameter, score_final)

output_path = 'C:/Users/bolub/OneDrive/Masaüstü/Ders/Year_4/FYP/'

with open(output_path, 'w') as out:
    writer = csv.writer(out)
    for row in output_rows:
        writer.writerow(row)
out.close()
